# Pulp

Using pulp to solve linear programming questions.

[The Beer Distribution Problem for the PuLP Modeller](https://coin-or.github.io/pulp/CaseStudies/a_transportation_problem.html)

Authors: Antony Phillips, Dr Stuart Mitchell  2007

In [1]:
from pulp import *

The start of the formulation is a simple definition of the nodes and their limits/capacities. The node names are put into lists, and their associated capacities are put into dictionaries with the node names as the reference keys:

In [2]:
# Creates a list of all the supply nodes
Warehouses = ["A", "B"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000, "B": 4000}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {
    "1": 500,
    "2": 900,
    "3": 1800,
    "4": 200,
    "5": 700,
}

The cost data is then inputted into a list, with two sub lists: the first containing the costs of shipping from Warehouse A, and the second containing the costs of shipping from Warehouse B.

The Warehouses and Bars lists (Supply and Demand nodes) are added to make a large list (of all nodes) and inputted into PuLPs makeDict function. The second parameter is the costs list as was previously created, and the last parameter sets the default value for an arc cost.

Once the cost dictionary is created, if costs[“A”][“1”] is called, it will return the cost of transporting from warehouse A to bar 1, 2. If costs[“C”][“2”] is called, it will return 0, since this is the defined default.


In [3]:
# Creates a list of costs of each transportation path
costs = [  # Bars
    # 1 2 3 4 5
    [2, 4, 5, 2, 1],  # A   Warehouses
    [3, 1, 3, 2, 3],  # B
]

# The cost data is made into a dictionary
costs = makeDict([Warehouses, Bars], costs, 0)

The prob variable is created using the *LpProblem* function, with the usual input parameters.

In [5]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Beer_Distribution_Problem", LpMinimize)

A list of tuples is created containing all the arcs.

In [6]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w, b) for w in Warehouses for b in Bars]

A dictionary called vars is created which contains the LP variables.

The reference keys to the dictionary are the warehouse name, then the bar name([“A”][“2”]) , and the data is Route_Tuple. (e.g. [“A”][“2”]: Route_A_2). The lower limit of zero is set, the upper limit of None is set, and the variables are defined to be Integers.

In [7]:
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None, LpInteger)

The objective function is added to the variable prob using a list comprehension. Since vars and costs are now dictionaries (with further internal dictionaries), they can be used as if they were tables, as for (w,b) in Routes will cycle through all the combinations/arcs. Note that i and j could have been used, but w and b are more meaningful.

In [8]:
# The objective function is added to 'prob' first
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

The supply and demand constraints are added using a normal for loop and a list comprehension.

Supply Constraints: For each warehouse in turn, the values of the decision variables (number of beer cases on arc) to each of the bars is summed, and then constrained to being less than or equal to the supply max for that warehouse. 

Demand Constraints: For each bar in turn, the values of the decision variables (number on arc) from each of the warehouses is summed, and then constrained to being greater than or equal to the demand minimum.

In [9]:
# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += (
        lpSum([vars[w][b] for b in Bars]) <= supply[w],
        "Sum_of_Products_out_of_Warehouse_%s" % w,
    )

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += (
        lpSum([vars[w][b] for w in Warehouses]) >= demand[b],
        "Sum_of_Products_into_Bar%s" % b,
    )

In [10]:
prob

Beer_Distribution_Problem:
MINIMIZE
2*Route_A_1 + 4*Route_A_2 + 5*Route_A_3 + 2*Route_A_4 + 1*Route_A_5 + 3*Route_B_1 + 1*Route_B_2 + 3*Route_B_3 + 2*Route_B_4 + 3*Route_B_5 + 0
SUBJECT TO
Sum_of_Products_out_of_Warehouse_A: Route_A_1 + Route_A_2 + Route_A_3
 + Route_A_4 + Route_A_5 <= 1000

Sum_of_Products_out_of_Warehouse_B: Route_B_1 + Route_B_2 + Route_B_3
 + Route_B_4 + Route_B_5 <= 4000

Sum_of_Products_into_Bar1: Route_A_1 + Route_B_1 >= 500

Sum_of_Products_into_Bar2: Route_A_2 + Route_B_2 >= 900

Sum_of_Products_into_Bar3: Route_A_3 + Route_B_3 >= 1800

Sum_of_Products_into_Bar4: Route_A_4 + Route_B_4 >= 200

Sum_of_Products_into_Bar5: Route_A_5 + Route_B_5 >= 700

VARIABLES
0 <= Route_A_1 Integer
0 <= Route_A_2 Integer
0 <= Route_A_3 Integer
0 <= Route_A_4 Integer
0 <= Route_A_5 Integer
0 <= Route_B_1 Integer
0 <= Route_B_2 Integer
0 <= Route_B_3 Integer
0 <= Route_B_4 Integer
0 <= Route_B_5 Integer

In [11]:
status = prob.solve()

In [12]:
print(f"status: {prob.status}, {LpStatus[prob.status]}")

status: 1, Optimal


In [14]:
print(f"objective: {prob.objective.value()}")

objective: 8600.0


In [15]:
for var in prob.variables():
    print(f"{var.name}: {var.value()}")

Route_A_1: 300.0
Route_A_2: 0.0
Route_A_3: 0.0
Route_A_4: 0.0
Route_A_5: 700.0
Route_B_1: 200.0
Route_B_2: 900.0
Route_B_3: 1800.0
Route_B_4: 200.0
Route_B_5: 0.0


In [16]:
for name, constraint in prob.constraints.items():
    print(f"{name}: {constraint.value()}")

Sum_of_Products_out_of_Warehouse_A: 0.0
Sum_of_Products_out_of_Warehouse_B: -900.0
Sum_of_Products_into_Bar1: 0.0
Sum_of_Products_into_Bar2: 0.0
Sum_of_Products_into_Bar3: 0.0
Sum_of_Products_into_Bar4: 0.0
Sum_of_Products_into_Bar5: 0.0


Also, refer to this [website](https://realpython.com/linear-programming-python/) for more information on how to use pulp.